Basic classification of stadsarchief images

In [ ]:
import numpy as np
import math
from PIL import Image
from scipy import misc
import keras.backend as K
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

import keras
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.initializers import RandomUniform
from keras.layers import LeakyReLU
from keras.preprocessing.image import ImageDataGenerator
 

from stats import list_stats, show_train_curves, show_prediction_list, show_prediction_images
# from stats import show_train_curves
from data import load_data, split_data
from image_display import show_image

# Hot reload packages
%load_ext autoreload
%autoreload 2


# Load dataset

In [ ]:
img_dim = (200, 200, 3);
img_dim = (250, 250, 3);
# img_dim = (300, 300, 3);
# img_dim = (400, 400, 3);
IMG_DIR = f'examples/aanvraag_besluit/resized/{img_dim[0]}x{img_dim[1]}/'
LABEL_DIR = 'examples/aanvraag_besluit/labels/'


[X, Y_yaml, ids] = load_data(IMG_DIR, LABEL_DIR)
X = X / 255.  # normalize image data between 0 and 1
Y_type = np.array([y.get('type') for y in Y_yaml])

print(f"shape X: {X.shape}")
print(f"shape Y: {Y_type.shape}")
print(f"len ids: {len(ids)}")

def shuffle(X, Y, ids):
    assert isinstance(ids, list)
    indices = np.random.permutation(X.shape[0])
    X_shuffled = X[indices]
    Y_shuffled = Y[indices]
    ids_shuffled = [ids[i] for i in indices]
    return (X_shuffled, Y_shuffled, ids_shuffled)

# print('shuffling X, Y and ids')
# X_shuffled, Y_shuffled, ids_shuffled = shuffle(X, Y_type, ids)
# X = X_shuffled
# Y_type = Y_shuffled
# ids = ids_shuffled

print(f"shape X: {X.shape}")
print(f"shape Y: {Y_type.shape}")
print(f"len ids: {len(ids)}")

list_stats(Y_type)

In [ ]:
# Convert string to idx
types = list(set(Y_type))
num_classes = len(types)
print(f"number of classes: {num_classes}")
print(f"types encountered: {types}")

def convert_to_index(Y, types):
    return np.array([types.index(y) for y in Y])

Yind = convert_to_index(Y_type, types)
print(Y_type[:3])
print(Yind[:3])

list_stats(Yind)

In [ ]:
Y = keras.utils.to_categorical(Yind, num_classes)  # convert class vectors to binary class matrices
print(Yind.shape)
print(Y.shape)
print(Y[:10])

In [ ]:
# Split to train and test
SPLIT = 0.8

# (X_train, Y_train), (X_test, Y_test) = split_data(X, Yscaled, SPLIT)
(X_train, Y_train, ids_train), (X_test, Y_test, ids_test) = split_data(X, Y, ids, SPLIT)

print("split data shapes:")
print(f"X_train: {X_train.shape}")
print(f"Y_train: {Y_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"Y_test: {Y_test.shape}")
print(f"ids_train: {len(ids_train)}")
print(f"ids_test: {len(ids_test)}")

# Define model

In [ ]:
# def build_model(num_classes, img_dim):
#     model = Sequential()
#     model.add(Conv2D(32, kernel_size=(3, 3),
#                      activation='relu',
#                      input_shape=img_dim))
#     model.add(Conv2D(64, (3, 3), activation='relu'))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Dropout(0.25))
#     model.add(Flatten())
#     model.add(Dense(128, activation='relu'))
#     model.add(Dropout(0.5))
#     model.add(Dense(num_classes, activation='softmax'))
#     return model


def build_model(num_classes, img_dim):
    print(img_dim)
    model = Sequential()
    model.add(Conv2D(16, kernel_size=(3, 3), activation='linear', input_shape=img_dim))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))

    model.add(Conv2D(32, (3, 3), activation='linear'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))
    
    model.add(Conv2D(32, (3, 3), activation='linear'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))
    
    model.add(Conv2D(32, (3, 3), activation='linear'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))
    
    model.add(Flatten())
    model.add(Dense(32, activation='linear', kernel_initializer = RandomUniform()))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dense(num_classes, activation='softmax'))
    return model


def build_medium_size(num_classes, img_dim):
    # src: https://www.learnopencv.com/image-classification-using-convolutional-neural-networks-in-keras/
    print(img_dim)
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=img_dim))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
 
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
 
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
 
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return model

model = build_model(num_classes, img_dim)
# model = build_vgg16(num_classes, img_dim)
# model = build_medium_size(num_classes, img_dim)
model.summary()

# Train

In [ ]:
batch_size = 200
epochs = 100

datagen = ImageDataGenerator(
        zoom_range=0.1,        # randomly zoom into images
        rotation_range=5,      # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1, # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,# randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False    # randomly flip images
)
 
    
def compile_model(model):
    assert(K.image_data_format() == 'channels_last')
    model.compile(
        loss=keras.losses.categorical_crossentropy,
#         optimizer=keras.optimizers.Adadelta(),
#         optimizer='rmsprop',
        optimizer=keras.optimizers.Adam(),        
        metrics=['accuracy']
    )

def train(model, X_train, Y_train, X_test, Y_test, batch_size, epochs):
    compile_model(model)
    history = model.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(X_test, Y_test)
           )
    return history

def train_gen(model, X_train, Y_train, X_test, Y_test, batch_size, epochs):
    compile_model(model)
    history = model.fit_generator(
        datagen.flow(X_train,
                     Y_train,
                     batch_size=batch_size
        ),
        steps_per_epoch=int(np.ceil(X_train.shape[0] / float(batch_size))),
        epochs=epochs,
        validation_data=(X_test, Y_test),
        workers=4
    )
    return history
history = train(model, X_train, Y_train, X_test, Y_test, batch_size, epochs)
# history = train_gen(model, X_train, Y_train, X_test, Y_test, batch_size, epochs)


show_train_curves(history)

In [ ]:
score = model.evaluate(X_train, Y_train, verbose=1)
print('Train loss:', round(score[0], 3))
print(f'Train accuracy: {round(score[1] * 100, 2)}%')

score = model.evaluate(X_test, Y_test, verbose=1)
print('Test loss:', round(score[0], 3))
print(f'Test accuracy: {round(score[1] * 100, 2)}%')


In [ ]:
print(f"types: {types}")

print("train predictions, truth")
predictions_train =  model.predict(X_train, verbose=1)
show_prediction_list(predictions_train, Y_train)

print("test predictions, truth")
predictions_test = model.predict(X_test, verbose=1)
show_prediction_list(predictions_test, Y_test)

In [ ]:
idx = 11
print(ids_test[idx])
show_image(X_test, idx)

In [ ]:
Y_train_idx = np.argmax(Y_train, axis=1)        
Y_test_idx = np.argmax(Y_test, axis=1)

print("train set:")
show_prediction_images(X_train, Y_train_idx, ids_train, predictions_train, types, 10)

print("test set:")
show_prediction_images(X_test, Y_test_idx, ids_test, predictions_test, types, 50)